In [57]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np

# Create sliders for Effective Slope, Compound Slope, and Distance
effective_slope = widgets.FloatSlider(value=6, min=0, max=30, step=0.5, description='Average Slope (100m) (°)')
compound_slope = widgets.FloatSlider(value=7, min=0, max=20, step=0.5, description='Compound Slope (°)')  # Capped at 20 degrees
distance = widgets.FloatSlider(value=30, min=0, max=30, step=1, description='Distance (m)')


# Function to calculate weighted slope based on Table 1 data
def calculate_weighted_slope(eff_slope, comp_slope, comp_dist):
    weight = 0  # Set a default value for weight to avoid unassigned reference
    if eff_slope == comp_slope:
        return eff_slope  # No additional weight if slopes are equal

    # Determine the weight based on compound distance and slope ranges from Table 1
    if comp_dist <= 10:
        weight = 0
    elif 10 < comp_dist <= 20:
        if 0 <= comp_slope <= 5:
            weight = 0.5
        elif 5 < comp_slope <= 10:
            weight = 1
        elif 10 < comp_slope <= 15:
            weight = 1.5
        elif 15 < comp_slope <= 20:
            weight = 2
    elif 20 < comp_dist <= 30:
        if 0 <= comp_slope <= 5:
            weight = 2
        elif 5 < comp_slope <= 10:
            weight = 2
        elif 10 < comp_slope <= 15:
            weight = 3
        elif 15 < comp_slope <= 20:
            weight = 4
    else:
        weight = 0  # Default to no adjustment if out of range

    return eff_slope + weight

# Function to update and display the graph
def update_slope_graph(eff_slope, comp_slope, comp_dist):
    final_slope = calculate_weighted_slope(eff_slope, comp_slope, comp_dist)

    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot Compound Slope line for the first segment (0 to comp_dist)
    ax.plot([0, comp_dist], [0, -comp_slope], 'r-', lw=3, label=f'Compound Slope: {comp_slope}°')

    # Continue the Slope 1 line from the end of the compound slope (keep the line but remove from legend)
    ax.plot([comp_dist, 100], [-comp_slope, -(comp_slope + (eff_slope * (100 - comp_dist)) / 100)], 'b-', lw=3)

    # Add black dotted line from the start of the house to the end of the slope
    ax.plot([0, 100], [0, -(comp_slope + (eff_slope * (100 - comp_dist)) / 100)], 'k--', lw=2, label='Average slope (100m)')

    # Display house icon
    ax.text(-20, 0, 'House', fontsize=20, color='black', va='center')

    # Annotate the compound slope with better spacing
    ax.text(comp_dist / 2, -comp_slope * 1.2, f'{comp_slope}°', fontsize=14, color='red', weight='bold')  # Lowered red text position

    # Annotate the slope above the black dotted line and change color to black
    ax.text((100 - comp_dist) / 2 + comp_dist, -(comp_slope + eff_slope / 2) + 5, f'{eff_slope}°', fontsize=14, color='black', weight='bold')

    # Annotate the final slope result (effective slope + weighted slope)
    ax.text(70, -25, f'Final Slope: {final_slope:.1f}°', fontsize=16, color='green', weight='bold')

    # Set the limits for the graph
    ax.set_ylim(-30, 10)
    ax.set_xlim(0, 100)

    # Add distance markers every 10 meters
    ax.set_xticks(np.arange(0, 110, 10))

    # Labels and title
    ax.set_xlabel('Distance (m)', fontsize=12)
    ax.set_ylabel('Slope (degrees)', fontsize=12)
    ax.set_title('Effective and Compound Slope Visualisation', fontsize=14)

    # Legend (the blue line is not included)
    ax.legend(loc='lower left')

    ax.grid(True)
    plt.show()

# Link the sliders to the update function
widgets.interactive(update_slope_graph, eff_slope=effective_slope, comp_slope=compound_slope, comp_dist=distance)


interactive(children=(FloatSlider(value=6.0, description='Average Slope (100m) (°)', max=30.0, step=0.5), Floa…